# Shopee Product Matching
* [Mô tả bài toán](#section-1)
* [Phân tích dữ liệu](#section-2)
* [Convert, clean dữ liệu](#section-3)
* [Thuật toán mô hình được chọn](#section-4)
* [Kết quả thu được](#section-5)
* [Kết quả khi submit](#section-4)
 

<a id="section-1"></a>
# Mô tả bài toán
Tìm các sản phẩm trùng lặp đóng vai trò rất quan trọng trong sự cạnh tranh của các công ty thương mại điện tử. <br>
Hai hình ảnh khác nhau cùng một kho có thể là cùng một sản phẩm hoặc hai sản phẩm hoàn toàn tách biệt. Ta cần dự đoán những sản phẩm nào là cùng một nhóm giống nhau. <br>
Tập dữ liệu đầu vào là một file csv có chứa những thông tin sau:
* `posting_id` - Mã ID của bài đăng 
* `image` - Hình ảnh của sản phẩm(Mã md5sum)
* `image_phash` - Mã perceptual hash của sản phẩm
* `title` - tiêu đề của sản phẩm
* `label_group` - Mã ID của của các sản phẩm cùng loại(không chứa trong bộ dữ liệu test) 
<br>
<br>
Tập đầu ra là các sản phẩm liên quan đến bài đăng được đưa ra file csv và có định dạng sau:
* `posting_id` - mã ID của bài đăng
* `matches` Các sản phẩm có cùng mã ID bài đăng. Số lượng sản phẩm trong mỗi nhóm cần tìm không vượt quá 50 và các bài đăng luôn trùng với chính nó

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
!pip install stylecloud
!pip install editdistance

In [ ]:
import numpy as np 
import pandas as pd 

import math
import random 
import os 
import cv2
import timm
import string
import cv2, matplotlib.pyplot as plt

from tqdm import tqdm 

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import torch 
from torch.utils.data import Dataset 
from torch import nn
import torch.nn.functional as F 

import gc
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

## Khởi tạo môi trường

In [ ]:
class CFG:
    
    img_size = 512
    batch_size = 12
    seed = 2021
    
    device = 'cuda'
    classes = 11014
    
    scale = 30 
    margin = 0.5

In [ ]:
def seed_torch(seed=2021):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch(CFG.seed)

## Đọc dữ liệu

In [ ]:
def read_dataset(name="train"):
    df = pd.read_csv('../input/shopee-product-matching/{}.csv'.format(name))
    df_cu = cudf.read_csv('../input/shopee-product-matching/{}.csv'.format(name))
    image_paths = '../input/shopee-product-matching/{}_images/'.format(name) + df['image']
    return df, df_cu, image_paths

df_train,df_cu_train,image_paths_train = read_dataset(name="train")
df,df_cu,image_paths = read_dataset(name="test")
len_d = len(df_cu)

<a id="section-2"></a>
# Phân tích dữ liệu
## In ra 5 hàng dầu tiên của bộ dữ liệu train

In [ ]:
df_train.head()

> Trường `image` chứa đuôi .jpg là file dẫn đến tệp hình ảnh

## In ra những hàng đầu tiên của bộ test

In [ ]:
df.head()

## In ra các sản phẩm ngẫu nhiên bộ train ,test

In [ ]:
WORKING_DIR = '../input/shopee-product-matching/'
def displayImgDF(df, random=False, COLS=6, ROWS=4, train=True):
    for k in range(ROWS):
        plt.figure(figsize=(20,5))
        for j in range(COLS):
            if random: row = np.random.randint(0,len(df))
            else: row = COLS*k + j
            name = df.iloc[row,1]
            title = df.iloc[row,3]
            title_return = ""
            for i,ch in enumerate(title):
                title_return += ch
                if (i != 0) and (i % 20 ==0 ): title_return += '\n' # Cắt các title quá dài
            if train: path = 'train_images/'
            else: path = 'test_images/'
            img = cv2.imread('../input/shopee-product-matching/' + path + name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.subplot(1,COLS,j+1)
            plt.title(title_return)
            plt.axis('off')
            plt.imshow(img)
        
        plt.show()
        
print('Bộ dữ liệu train')        
displayImgDF(df_train,random=True)
print('Bộ dữ liệu test')
displayImgDF(df, random=False, COLS=3, ROWS=1, train=False)

## WordCloud

In [ ]:
import stylecloud
stylecloud.gen_stylecloud(text=' '.join(df_train['title']),
                          icon_name='fas fa-shopping-cart',
                          palette='colorbrewer.qualitative.Accent_8',
                          background_color='black',
                          gradient='horizontal',
                          size=1024)

from IPython.display import Image
Image(filename="./stylecloud.png", width=604, height=604)

> Từ wordcloud trên có thể thấy bộ dữ liệu chủ yếu là tiếng Indonesia và 1 phần tiếng Anh

## Phân bố độ dài các tiêu đề

In [ ]:
import plotly.express as px
df_train['Độ dài tiêu đề sản phẩm'] = df_train['title'].apply(lambda x: len(x))
def plot_distribution(x, title):

    fig = px.histogram(
    df_train, 
    x = x,
    width = 800,
    height = 500,
    title = title
    )
    
    fig.show()
plot_distribution(x = 'Độ dài tiêu đề sản phẩm', title = 'Phân bố độ dài tiêu đề sản phẩm')


> Có thể thấy độ dài các tiêu đề sản phẩm tập trung nhiều nhất vào khoảng 30-50. Số ít có độ dài hơn 100

## Hiển thị các sản phẩm trùng nhau

In [ ]:
import editdistance
def plot_edit_distance(df):
    x_axis = []
    y_axis = []
    root_title = df.iloc[0, 3]
    len_df = len(df)
    for i in range(1, len_df):
        x_axis.append(i)
        y_axis.append(editdistance.eval(root_title, df.iloc[i, 3]))
    plt.plot(x_axis, y_axis)
    plt.xlabel("Chỉ số tiêu đề")
    plt.ylabel("Khoảng cách edit distance")
    plt.show()
    


In [ ]:
groups = df_train.label_group.value_counts()
top = df_train.loc[df_train.label_group==groups.index[0]]
displayImgDF(top, random=False, ROWS=2, COLS=4)

## Edit Distance của các sản phẩm Top 1 trùng nhau

In [ ]:
plot_edit_distance(top)

In [ ]:
top = df_train.loc[df_train.label_group==groups.index[1]]
displayImgDF(top, random=False, ROWS=2, COLS=4)

## Edit Distance của các sản phẩm Top 2 trùng nhau

In [ ]:
plot_edit_distance(top)

> Nhận xét:  Dựa trên quá trình phân tích trên, các sản phẩm cùng loại có thể có tiêu đề không liên quan nhiều đến nhau. Hình ảnh lại có thể được xác định một cách rõ ràng hơn. Tuy vậy, nhiều hình ảnh với các góc chụp khác nhau có thể gây khó khăn cho mô hình. Kết hợp giữa hình ảnh và tiêu đề rõ ràng cho một kết quả chính xác hơn

<a id="section-3"></a>
# Convert, clean dữ liệu
Từ quá trình phân tích trên có thể thấy tiêu đề các sản phẩm chứa nhiều các `punctuation` như `[|*`. Nhiều sản phẩm khá giống nhau về tiêu đề, nhưng có tiêu đề in hoa, có tiêu đề không. Vì vậy, để quá trình nhận dạng hiệu quả hơn cần convert về chữ thường và bỏ các `punctuation` <br>
Ngoải ra, các `stopword` - những từ phổ biến đã được trình bày ở wordcloud có thể gây ra nhiễu và cần được loại bỏ

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
def remove_stopwords(): # loại bỏ các stopword
    stop_words = set(stopwords.words('indonesian'))
    filtered_sentence = []
    for i in range(len_d):
        word_tokens = word_tokenize(df_cu['title'][i])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        df_cu['title'][i] = ' '.join(filtered_sentence)
def lower_text(): # convert về chữ thường
    for i in range(len_d):
        df_cu['title'][i] = df_cu['title'][i].lower()
        
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(): # loại bỏ các punctuation
    for i in range(len_d):
        df_cu['title'][i] = df_cu['title'][i].translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    

In [ ]:
print(df_cu['title'].head())
lower_text()
remove_punctuation()
remove_stopwords()
print('Sau khi được convert, clean')
print(df_cu['title'].head())

## Tạo dataset

In [ ]:
def get_test_transforms():

    return A.Compose(
        [
            A.Resize(CFG.img_size,CFG.img_size,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

In [ ]:
class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

<a id="section-4"></a>
# Thuật toán mô hình được chọn
## Additive Angular Margin Loss
### Giới thiệu
https://arxiv.org/pdf/1801.07698.pdf <br>
Hàm lỗi `softmax` truyền thống không có khả năng tối ưu các vector đặc trưng để làm tăng sự tương đồng giữa các đối tượng cùng lớp. Nói cách khác, hàm lỗi chưa làm được việc tăng sự khác biệt giữa các lớp. Ta có thể thấy qua hình minh họa sau:
![](https://i0.wp.com/s1.uphinh.org/2022/01/07/ml1.png) <br>
Hàm `softmax` phân tách được các vector đặc trưng nhưng lại mập mờ giữa các ranh giới quyết định(decison boundaries). Trong khi, `ArcFace` lại phân biệt rõ ràng giữa các lớp gần nhau
### Các bước tiến hành
Dựa trên đặc trưng $x_i$ và trọng số $W$ sau khi được chuẩn hóa, chúng ta có được giá trị $cos_{\theta_j}$(logit) được tính theo công thức sau:
$$\begin{align}
\large cos_{\theta_j} = W_j^Tx_i
\end{align}$$
Bước chuẩn hóa đặc trưng và trọng số khiến cho việc dự đoán chỉ phụ thuộc vào góc giữa chúng. Các vector đặc trưng được học sau đó sẽ được phân bố trên một hình cầu với bán kính $s$. Chính vì được phân bố như vậy, chúng ta sẽ thêm một hệ số $m$ vào góc giữa $x_i$ và $W_{y_i}$ để tăng sự tương đồng trong nội bộ lớp và sự khác biệt giữa các lớp. Sau đó, tính giá trị $cos(\theta_{y_i}+m)$ và nhân tất cả giá trị logit với hệ số $s$. Các logits lại được đưa vào hàm `softmax` và hàm lỗi `cross entropy` <br>
Cuối cùng ta thu được hàm lỗi:
$$\begin{align}
\large L_3=-\frac{1}{n}\sum_{i=1}^n log\frac{e^{s \thinspace cos\theta_{y_i}}}{e^{s \thinspace cos\theta_{y_i}}+\sum_{j=1,j\#{y_i}}^n}e^{s \thinspace cos\theta_{y_i}}
\end{align}$$
Piepline của toàn bộ quá trình như sau:
![](https://i0.wp.com/s1.uphinh.org/2022/01/07/ml2.png)


In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale # hệ số s
        self.margin = margin # hệ số m
        self.ls_eps = ls_eps 
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin) 
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight)) # tính cos(phi_j)
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2)) #tính sin(phi_j)
        phi = cosine * self.cos_m - sine * self.sin_m # tính góc giữa x_i và Wy_i
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale # nhân với hệ số s

        return output

## Model
Bài toán được giải dựa trên 2 mạng NFNet và EfficientNet <br>
#### Tại sao lại là NFNet và EfficientNet?
Bộ test có gần **70000** dữ liệu và thời gian chạy GPU submit bị giới hạn 2 tiếng. Vì vậy, đòi hỏi phải có các model hiệu quả về mặt thời gian nhưng cũng phải đáp ứng cao về performance. NFNet và EfficientNet đều tỏ ra tối ưu về thời gian huấn luyện cũng như suy luận. Về độ chính xác, EfficentNet đạt được kết quả tốt hơn so với các mạng ConvNet trước đó. Cỵ thể, mô hình đạt được 84.3% top-1 accuracy trong bộ dữ liệu ImageNet và nhỏ hơn 8.4 lần và suy luận nhanh hơn 6.1 lần các mạng Conv. NFNet thậm chí còn nhanh hơn 8.7 lần EfficientNet khi huấn luyện và đạt được 86.5% top-1 accuracy ImageNet. Cụ thể hơn, các model được sử dụng là `EfficentNetB3`, `EfficentNetB5` và `NFNetL0` đã được pretrain dựa trên hàm lỗi ArcMargin
## EfficientNet
https://arxiv.org/pdf/1905.11946.pdf <br>
Các tác giả có nhận xét cân bằng được các yếu tố độ sâu mạng, chiều rộng và độ phân giải cho ra một mô hình tốt hơn. Dựa trên quan sát đó, ta có thể scale chiều sâu/chiều rông/độ phân giải bằng một hệ số được gọi là `compound coefficient`
#### Compound Scaling
Compound Scaling sử dụng hệ số compound coefficient $\phi$ để scale đều độ sâu, rộng và phân giải theo công thức sau:
$$\begin{align}
\textrm{độ sâu} &: d=\alpha^{\phi} \\
\textrm{độ rộng} &: w=\beta^{\phi} \\
\textrm{độ phân giải} &: r=\gamma^{\phi} \\
\textrm{với} &: \alpha.\beta^2.\gamma^2 \approx 2 \\
& \alpha \ge 1, \beta \ge 1, \gamma \ge 1
\end{align}$$
<br>
$\alpha, \beta, \gamma$ là hằng số và có thể xác định bằng các thuật toán tìm kiếm lưới(grid search). $\phi$ giúp kiểm soát số lượng tài nguyên(resources) sẵn có cho model scaling, trong khi  $\alpha, \beta, \gamma$ mô tả lượng tài nguyên thêm vào độ rông, sâu , phân giải. Dựa trên các ý tương trên, mạng cơ sở `EfficientNet-B0` đã được xây dựng để tối ưu về cả performance và FLOPS <br>
Kiến trúc của mạng `EfficentNet-B0`: <br>
![](https://i.upanh.org/2022/01/08/ml3.png) <br>
Từ mạng cơ sở `EfficentNet-B0` được scale up với các hệ số $\phi$ khác nhau và thu được các mô hình từ B1 đến B7
### NFNet
https://arxiv.org/pdf/2102.06171.pdf <br>
NFNet được cải tiến dựa trên ResNet với các điểm khác biệt sau:
- Sửa đối các nhánh residual và tích chập với các trọng số được scale <br>
    - Để train các mạng ResNet sâu mà không cần Batch Normalization, cần bóp các hàm kích hoạt tại mỗi nhánh thặng dư. NFNet dùng 2 scalar $\alpha$ và $\beta$ để thực hiện điều đó
![](https://i.upanh.org/2022/01/08/ml4.png)    
- Adaptive Gradient Clipping
    - AGC được sử dụng để train mô hình với batch size và learning rate lớn 
- Tối ưu kiến trúc để thu được mô hình có độ chính xác cao và tăng tốc độ training
    - Mặc dù, sửa đối các nhánh thặng dư, tích chập hay thêm AGC nhưng model vẫn chưa đạt được kết quả cao như EfficientNet. Các tác giả sử dụng SE-ResNeXt-D làm mô hình cơ sở và thay đổi để đạt được performance cao

In [ ]:
class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = None,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc
        
        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim
        # load các trọng số ArcFace đã được pretrain
        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x

In [ ]:
def get_model(model_name = None, model_path = None, n_classes = None):
    model = ShopeeModel(model_name = model_name)
    model.eval()
    model.load_state_dict(torch.load(model_path))
    model = model.to(CFG.device)
    return model 

## Ensemble Learning
Các mô hình đề cập phía trên sẽ được kết hợp lại và lấy trung bình cộng

In [ ]:
class EnsembleModel(nn.Module):
    
    def __init__(self):
        super(EnsembleModel,self).__init__()
        self.m1 = get_model('eca_nfnet_l0','../input/shopee-pytorch-models/arcface_512x512_nfnet_l0 (mish).pt')
        self.m2 = get_model('tf_efficientnet_b5_ns','../input/shopee-pytorch-models/arcface_512x512_eff_b5_.pt')
        self.m3 = get_model('tf_efficientnet_b3_ns','../input/shopee-pytorch-models/arcface_512x512_eff_b3.pt')
    def forward(self,img,label):
        feat1 = self.m1(img, label)
        feat2 = self.m2(img, label)
        feat3 = self.m3(img, label)
    
        return (feat1 + feat2 + feat3) / 3

## Trích xuất các đặc trưng từ ảnh

In [ ]:
def get_image_embeddings(image_paths, model_name = None, model_path = None):
    embeds = []
    
    model = EnsembleModel()
    
    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [ ]:
image_embeddings = get_image_embeddings(image_paths.values)
print('Kết thúc image embeddings')

## Matching các đặc trưng ảnh dựa trên KNN
- 1. Chọn $k$ ảnh có đặc trưng gần nhất với ảnh đang xét. 
- 2. Nếu kết quả trả về ít hơn 2, nâng ngưỡng lên nhằm tránh việc không thu được kết quả nào thỏa mãn. Tuy nhiên do ngưỡng được tăng lên, có thể thu được nhiều kết quả không chính xác vì vậy chỉ dừng lại lấy 2 đặc trưng gần nhất. 
- 3. Ngược lại, kết quả trả về nhiều hơn 2 ảnh, giảm ngưỡng đi 1 khoảng `0.08888`. Nếu kết quả trả về nhỏ hơn 2 quay lại bước 2

In [ ]:
def get_image_predictions(df, embeddings,threshold = 0.0):
    
    if len(df) > 3: # bộ test là bộ đang submit
        KNN = 50 # do giới hạn của cuộc thi tối đa là 50 matching
    else : 
        KNN = 3
    
    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    predictions = []
    for k in tqdm(range(embeddings.shape[0])):
        idx = np.where(distances[k,] < threshold)[0] #lấy các chỉ số của các khoảng cách nhỏ hơn ngưỡng
        ids = indices[k,idx] # tìm các id dựa trên chỉ số
        posting_ids = df['posting_id'].iloc[ids].values
        if len(posting_ids) >= 2: # kết quả trả về lớn hơn 2
            idx_s = np.where(distances[k,] < threshold - 0.08888)[0] # giảm ngưỡng
            ids_s = indices[k, idx_s] 
            posting_ids_b = df['posting_id'].iloc[ids_s].values
            if len(posting_ids_b) >= 2:
                predictions.append(posting_ids_b) # chọn theo ngưỡng đã giảm
            else:
                predictions.append(posting_ids) # chọn theo ngưỡng ban đầu
        else:
            idx = np.where(distances[k,] < 0.51313)[0]
            ids = indices[k,idx]
            posting_ids = df['posting_id'].iloc[ids].values
            predictions.append(posting_ids[:2]) # chỉ lấy 2 ảnh gần nhất
            
        
    del model, distances, indices
    gc.collect()
    return predictions

## Matching các đặc trưng text dựa trên cosine similarity
Các `title` sẽ được Tfidf xử lý thành các vector đặc trưng. Sau đó chia thành các chunk nhỏ. Từng vector đặc trưng trong các chunk được tính cosine similarity với toàn bộ các vector đang có. Tuy nhiên, ngược với matching ảnh do cosine similarity càng lớn 2 vector đặc trưng càng giống nhau(cosine distance bằng 0). Các phép toán so sánh sẽ ngược với phần trên nhưng toàn bộ quá trình chọn ngưỡng vẫn tương tự như vậy

In [ ]:
def get_text_predictions(df_cu, max_features = 25_000):
    
    model = TfidfVectorizer(binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024 * 4 # chunk size

    num_chunk = len(df) // CHUNK # số lượng chunk
    if len(df) % CHUNK != 0: 
        num_chunk += 1
    for j in range(num_chunk):

        a = j * CHUNK # chỉ số bắt đầu chunk j
        b = (j + 1) * CHUNK # chỉ số kết thúc chunk j
        b = min(b, len(df))
        print('chunk', a, 'to', b)

        cos_similarity = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T # tính cosine similarity

        for k in range(b - a):
            indexes = cupy.where(cos_similarity[k,] > 0.7705)[0]
            ids = df_cu.iloc[cupy.asnumpy(indexes)].posting_id.to_pandas().values
            if len(ids) >= 2:
                indexes_lower_thres = cupy.where(indexes[k,] > 0.80105)[0]
                ids_lower_thres = df_cu.iloc[cupy.asnumpy(indexes_lower_thres)].posting_id.to_pandas().values
                if len(o) >= 2:
                    preds.append(ids_lower_thres)
                else:
                    preds.append(ids)
            else:
                indexes = cupy.where(cos_similarity[k,]>0.6555)[0]
                ids = df_cu.iloc[cupy.asnumpy(indexes)].posting_id.to_pandas().values
                preds.append(ids[:2])
    
    del model,text_embeddings
    gc.collect()
    return preds

## Image, Text Prediction

In [ ]:
image_predictions = get_image_predictions(df, image_embeddings, threshold = 0.36)
text_predictions = get_text_predictions(df_cu, max_features = 25_000)
print('Kết thúc image + text prediction')

## Kết hợp Text, Image prediction 

In [ ]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions']])
    return ' '.join(np.unique(x))

**Pipeine của toàn bộ quá trình được tóm gọn bằng sơ đồ sau** 
<br>
<br>
![so_do](https://i.upanh.org/2022/01/08/Untitled-Diagram.drawio1.png)

<a id="section-5"></a>
# Kết quả thu được

In [ ]:
df['image_predictions'] = image_predictions
df['text_predictions'] = text_predictions
df['matches'] = df.apply(combine_predictions, axis = 1)
df[['posting_id', 'matches']].to_csv('submission.csv', index = False)
df[['posting_id', 'matches']].head()
###out put the result

> Bộ test khi chưa submit chỉ có 3 hàng dữ liệu, vì vậy khi chạy đều cho ra kết quả chính xác. Model đã được pretrain với bộ train, khi chạy bộ train sẽ cho ra kết quả không khách quan

<a id="section-6"></a>
# Kết quả khi submit
![](https://i.upanh.org/2022/01/08/ml6.png)